In [1]:
from new_SiameseNet import *

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, CuDNN not available)
/home/ubuntu/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
# Prepare data
print 'Getting CIFAR10 data...'

data = get_CIFAR10_data()

x_train, y_train = data['X_train'], data['y_train']
x_val,   y_val   = data['X_val'],   data['y_val']

N = x_train.shape[0]

Getting CIFAR10 data...


In [3]:
# Specify structure of Siamese part of SiameseNet
# This part needs to be improved. I'm kind of just using random layers.
init = 'glorot_uniform'
in_shp = (3,32,32)
seq = Sequential()
seq.add(BatchNormalization(epsilon=1e-7,
                            mode=0,
                            axis=1,
                            momentum=0.9,
                            weights=None,
                            input_shape=in_shp))
#seq.add(Flatten())
#seq.add(Dense(128, activation='relu'))
#seq.add(Dropout(0.1))
#seq.add(Dense(128, activation='relu'))
#seq.add(Dropout(0.1))
for _ in range(1):
    seq.add(Convolution2D(10, 3, 3, init=init, border_mode='same'))
    seq.add(Activation('relu'))
#seq.add(Dense(128, activation='relu'))
seq.add(Flatten())
seq.add(Dense(128))
layers = seq

sn = SiameseNet(layers, input_shape=(3, 32, 32), verbose=True)
sn.compile()

Constructed a SiameseNet.
Successfully compiled the SiameseNet.


In [5]:
def _train_sn(sn, x_train, y_train, x_val, y_val, filepath):
    d_val = invert_dataset(x_val,  y_val)
    d_train = invert_dataset(x_train,  y_train)
    num_ep = 2
    history = sn.fit(*generate_data(x_train, d_train), 
           validation_data=generate_data(x_val, d_val),
           nb_epoch=num_ep)
    return history

history = _train_sn(sn, x_train, y_train, x_val, y_val, filepath='weights.h5')

Train on 97600 samples, validate on 1540 samples
Epoch 1/2
97600/97600 [==============================] - 54s - loss: 0.9045 - val_loss: 0.2265
Epoch 2/2
97600/97600 [==============================] - 54s - loss: 0.2272 - val_loss: 0.2242
Done training the SiameseNet.


In [7]:
#loss = sn.evaluate(*generate_data(d_val, examples_per_image=5))
d_val = invert_dataset(x_val,  y_val)
val_x_dat, val_y_dat = generate_data(x_val, d_val)
prediction = sn.predict(val_x_dat)[SiameseNet.OUTPUT]

ret_preds = prediction
max_d = np.max(ret_preds)
min_d = np.min(ret_preds)

print max_d
print min_d

#thresh = (max_d + min_d) / 2.0
#preds = [0,0]
#for i,p in enumerate(prediction):
#    if ret_preds[i] > thresh:
#        preds[1] += 1
#    else:
#        preds[0] += 1
#print preds

Predicted probabilities are {'output': array([[ 0.73530477],
       [ 0.50071985],
       [ 0.28664848],
       ..., 
       [ 0.70548493],
       [ 0.5286566 ],
       [ 0.5686509 ]])}
1.52992796898
0.235659629107


In [8]:
print history.history
l = history.history['loss']
val_l = history.history['val_loss']

{'loss': [0.90445654848559953, 0.22720627847265024], 'val_loss': [0.2265235638850695, 0.22422721416919261], 'batch': [762, 762], 'size': [64, 64]}


In [9]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [10]:
plt.figure()
plt.plot(l, label='Training Loss')
plt.title('Siamese Network Training/Validation Loss vs. Epochs')
plt.plot(val_l, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('siam_loss.png')

In [11]:
def similarity(sn, x1, x2):
    x = [x1, x2]
    prediction = sn.predict(x, batch_size=1)
    return prediction['output']